# 11. Optimal control of storage
In this notebook we use the previously developed demand forecasting model together with the
`optimisation.csv` data to compute an optimal 24-hour battery schedule.

We consider:
- PV system with maximum 5 kW power,
- Home battery with 10 kWh energy capacity and 5 kW charge/discharge limits,
- Grid connection with +-5 kW power limit.

Two PV scenarios are analysed:
- PV_low
- PV_high

The goal is to minimize the total electricity cost for the next 24 hours while respecting
all technical constraints. We compare the resulting schedules and total costs for the two PV cases.

## Setup and imports

In [1]:
from functions import *

## Load and preprocess the datasets

In [2]:
# train dataset
train_raw = load_data()
train_raw = train_raw.sort_index()
train_raw = train_raw.asfreq("h")
train_raw = train_raw.interpolate(method="time", limit_direction="both")
train_raw = train_raw.reset_index()
train_raw = train_raw.round(5)

forecast_raw = load_forecast_data()
forecast_raw = forecast_raw.sort_index()
forecast_raw = forecast_raw.asfreq("h")
forecast_raw = forecast_raw.interpolate(method="time", limit_direction="both")
forecast_raw = forecast_raw.reset_index()
forecast_raw = forecast_raw.round(5)

optimisation_raw = load_optimisation_data()
optimisation_raw = optimisation_raw.sort_index()
optimisation_raw = optimisation_raw.asfreq("h")
optimisation_raw = optimisation_raw.interpolate(method="time", limit_direction="both")
optimisation_raw = optimisation_raw.reset_index()
optimisation_raw = optimisation_raw.round(5)

show_table_info(train_raw, "Train data")
show_table_info(forecast_raw, "Forecast data")
show_table_info(optimisation_raw, "Optimisation data")


TRAIN DATA SUMMARY
Shape: 8,760 rows × 18 columns
Time span: 2013-07-01 00:00:00+00:00 -> 2014-06-30 23:00:00+00:00

                         Column    Type   NA %
                        pv_mod1 float64  0.00%
                        pv_mod2 float64  0.00%
                        pv_mod3 float64  0.00%
                         demand float64  0.00%
                             pv float64  0.00%
                          price float64  0.00%
                    temperature float64  0.00%
                 pressure (hPa) float64  0.00%
                cloud_cover (%) float64  0.00%
            cloud_cover_low (%) float64  0.00%
            cloud_cover_mid (%) float64  0.00%
           cloud_cover_high (%) float64  0.00%
          wind_speed_10m (km/h) float64  0.00%
     shortwave_radiation (W/m²) float64  0.00%
        direct_radiation (W/m²) float64  0.00%
       diffuse_radiation (W/m²) float64  0.00%
direct_normal_irradiance (W/m²) float64  0.00%


FORECAST DATA SUMMARY
Shape: 168 r

,Column,Type,NA %
0,pv_low,float64,0.00%
1,pv_high,float64,0.00%
2,price,float64,0.00%
3,temperature,float64,0.00%
4,pressure (hPa),float64,0.00%
5,cloud_cover (%),int64,0.00%
6,cloud_cover_low (%),int64,0.00%
7,cloud_cover_mid (%),int64,0.00%
8,cloud_cover_high (%),int64,0.00%
9,wind_speed_10m (km/h),float64,0.00%


## Add engineered features

In [3]:
train_fe = add_time_weather_features(train_raw)
forecast_fe = add_time_weather_features(forecast_raw)
optimisation_fe = add_time_weather_features(optimisation_raw)

In [4]:
FEATURES = [
    "hour_sin", "hour_cos", "is_weekend",
    "cooling_degree", "heating_degree",
    "temperature", "pressure (hPa)",
    "cloud_cover (%)", "wind_speed_10m (km/h)",
    "shortwave_radiation (W/m²)",
    "direct_radiation (W/m²)",
    "diffuse_radiation (W/m²)",
    "direct_normal_irradiance (W/m²)",
    "price",
]

train_all = pd.concat([train_fe, forecast_fe], ignore_index=True)

X_train = train_all[FEATURES]
y_train = train_all["demand"]

X_optimisation = optimisation_fe[FEATURES]

X_train.head()

,hour_sin,hour_cos,is_weekend,cooling_degree,heating_degree,temperature,pressure (hPa),cloud_cover (%),wind_speed_10m (km/h),shortwave_radiation (W/m²),direct_radiation (W/m²),diffuse_radiation (W/m²),direct_normal_irradiance (W/m²),price
0,0.000000,1.000000,0,0.0,4.5,13.5,1011.3,4.0,10.5,0.00000,0.0,0.0,0.0,0.01605
1,0.258819,0.965926,0,0.0,4.8,13.2,1010.8,27.0,11.9,0.00000,0.0,0.0,0.0,0.00095
2,0.500000,0.866025,0,0.0,4.9,13.1,1010.3,33.0,11.6,0.00000,0.0,0.0,0.0,0.00060
3,0.707107,0.707107,0,0.0,5.0,13.0,1010.3,28.0,11.2,51.45455,2.0,7.0,30.1,0.00046
4,0.866025,0.500000,0,0.0,4.2,13.8,1010.2,16.0,11.7,102.90909,30.0,31.0,252.0,0.00046


## Train XGBoost regression model for demand

In [5]:
xgb_model, xgb_history = train_xgboost(X_train, y_train)

optimisation_fe["demand_fc"] = xgb_model.predict(X_optimisation)

optimisation_fe[["timestamp", "demand_fc", "pv_low", "pv_high", "price"]].head()

,timestamp,demand_fc,pv_low,pv_high,price
0,2014-07-08 00:00:00+00:00,0.311417,0.0,0.0,0.06096
1,2014-07-08 01:00:00+00:00,0.408060,0.0,0.0,0.07006
2,2014-07-08 02:00:00+00:00,0.316114,0.0,0.0,0.07005
3,2014-07-08 03:00:00+00:00,0.287530,0.0,0.0,0.05500
4,2014-07-08 04:00:00+00:00,0.334048,0.0,0.0,0.05757


## Define system parameters and optimisation model

In [6]:
PV_CAP = 5.0          # kW, max PV power
BATT_CAP = 10.0       # kWh, battery energy capacity
BATT_POWER = 5.0      # kW, max charge/discharge power
GRID_LIMIT = 5.0      # kW, grid import/export limit
EFFICIENCY = 0.95     # round-trip efficiency (per step)

T = len(optimisation_fe)
hours = np.arange(T)
# 24 hours

print(f"Horizon: {T} hours")
print(f"Time span: {optimisation_fe['timestamp'].min()} -> {optimisation_fe['timestamp'].max()}")

Horizon: 24 hours
Time span: 2014-07-08 00:00:00+00:00 -> 2014-07-08 23:00:00+00:00


## Run optimisation for PV_low and PV_high

In [7]:
# Prepare series for optimisation

demand = optimisation_fe["demand_fc"].values
price_buy = optimisation_fe["price"].values
price_sell = 0.5 * price_buy  # assumption: sell price is 50% of buy price

pv_low = optimisation_fe["pv_low"]
pv_high = optimisation_fe["pv_high"]

print("Demand forecast range:", demand.min().round(3), "–", demand.max().round(3), "kW")
print("PV_low range:", pv_low.min().round(3), "–", pv_low.max().round(3), "kW")
print("PV_high range:", pv_high.min().round(3), "–", pv_high.max().round(3), "kW")


Demand forecast range: 0.288 – 1.25 kW
PV_low range: 0.0 – 0.29 kW
PV_high range: 0.0 – 2.83 kW


## Optimise for both scenarios


In [8]:
result_low = optimize_storage(demand, pv_low,  price_buy, price_sell)
result_high = optimize_storage(demand, pv_high, price_buy, price_sell)

print(f"PV_low cost:  €{result_low['cost']:.2f}  (status: {result_low['status']})")
print(f"PV_high cost: €{result_high['cost']:.2f}  (status: {result_high['status']})")


PV_low cost:  €0.77  (status: optimal)
PV_high cost: €0.04  (status: optimal)


In [9]:
plot_optimisation_profile(hours, demand, pv_low,  result_low,
                          title_prefix="PV_low",
                          filename="ex11_fig1_PV_low.svg")


In [10]:
plot_optimisation_profile(hours, demand, pv_high, result_high,
                          title_prefix="PV_high",
                          filename="ex11_fig2_PV_high.svg")

In [11]:
# Summary table
summary = compute_optimisation_summary(result_low, result_high)
summary.columns

Index(['Scenario', 'Total cost [€]', 'Energy bought (kWh)',
       'Energy sold (kWh)', 'Battery cycles', 'SOC min', 'SOC max'],
      dtype='object')

In [12]:
fig_cost = go.Figure()
summary_rounded = summary.round(3)
fig_cost.add_trace(go.Bar(
    x=summary["Scenario"],
    y=summary["Total cost (€)"],
    marker_color=ENERGY_COLORS["grid"]
))

fig_cost.update_layout(
    title="Total cost for PV_low and PV_high scenarios",
    xaxis_title="Scenario",
    yaxis_title="Total cost (€)",
    **PLOT_STYLE
)

save_fig_plotly(fig_cost, "ex11_fig3_cost_comparison.svg", width=800, height=500)
fig_cost.show()


KeyError: 'Total cost (€)'

## Conclusion
In this task we tested how a household battery should be used over the next 24 hours, using demand, PV and price forecasts. We compared two cases: low PV and high PV. The results show a clear pattern: with more solar energy available, the battery can charge earlier in the day, reduce grid imports, and lower total cost. In the low-PV case, the system depends much more on the grid, which makes the cost higher. The battery always follows a logical strategy, charging when energy is cheap or PV is available, and discharging when demand or prices are higher. Overall, the optimisation confirms that higher PV production leads to less grid usage, more self-consumption, and noticeably lower energy costs.